# IMDB Movie data scraping - 1000 movies

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getImdbLink(start):
    url = 'https://www.imdb.com/search/title/?groups=top_1000&start='+str(start)+ '&ref_=adv_nxt'
    return (url)

In [3]:
def getOnePageData(start):
    url = getImdbLink(start)
    results = requests.get(url)
    soup = BeautifulSoup(results.text, "lxml")
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    return (movie_div)

In [4]:
def getAllPageData(start):
    movie_divs = []
    
    while start <= 1001:
        movie_divs.append(getOnePageData(start))
        start = start + 50
    return movie_divs

In [6]:
def getMovies(movie_div):
    #our loop through each container
    for container in movie_div:
        title = container.h3.a.text
        titles.append(title)
        
        '''#year
        year = container.h3.find("span", class_="lister-item-year").text
        years.append(year)
        #time
        time = container.p.find("span", class_="runtime").text
        if(time):
            runtime.append(time)
        else:
            runtime.append('_')'''
            
           #year
        year = container.h3.find("span", class_="lister-item-year").text
        # to find only year digit from the text and convert into integer
        year1 = re.findall(r'\d+', year)
        years.append(int(year1[0]))
        
        #time
        time1 = container.p.find("span", class_="runtime").text
        time = re.findall(r'\d+', time1)
        if(time):
            runtime.append(int(time[0]))
        else :
            runtime.append(np.nan)    

        #rating
        rating= float(container.strong.text)
        imdb_ratings.append(rating)
       
        #metascores
        meta_score = container.find('span', class_='metascore').text.replace(' ', '') if container.find('span', class_='metascore') else None
    
        
        #metascore = container.find('span', class_= "metascore  favorable").text
        #meta_score = metascore if metascore else None
        metascores.append(meta_score)

        #vote
        nv = container.find_all('span', attrs={'name': 'nv'})
        vote = nv[0].text  if len(nv) > 0 else None
        vote1 = re.sub("[^\d]", "", vote)
        votes.append(int(vote1))

        #gross
        gross = re.sub(r'[$M]','', nv[1].text) if (len(nv) > 1) else None
        us_gross.append(gross)

        # get directors and stars
        people = container.find('p', class_= '').text
        cast = people.split('|')
        
        # Directors
        #Director = cast[0].split(':')[-1].strip('\n')
        #director_lists.append(Director)
        
    
        directors = ''
        director_list = cast[0].split(':')[-1].split(',')
        for director in director_list:
            directors += '"{}"'. format(director.strip(' ').strip('\n'))
        director_lists.append(directors)
        
        
        # stars
        stars = ''
        star_list = cast[1].split(':')[1].split(',')
        for star in star_list:
            stars += '"{}" '.format(star.strip(' ').strip('\n'))

        star_lists.append(stars)
        gern_re=''
        genre_list = container.find('span', class_= 'genre').get_text(strip=True)

        genres.append(genre_list)

In [5]:
# start movie from 1 -1000 increment by 50
start=1
#initiate data storage
movie_divs = getAllPageData(start)


In [7]:
#scrape data from stored all 1000 movies in movie_divs
titles = []
years = []
runtime = []
imdb_ratings = []
votes = []
metascores = []
us_gross = []
director_lists = []
star_lists = []
genres = []
for movie_div in movie_divs:
    getMovies(movie_div)

In [8]:
newMetalist = []
for item in metascores:
    newMetalist.append(item if item != '******' else None)
newMetalist

['67',
 '87',
 '81',
 '73',
 '84',
 '76',
 '95',
 '78',
 '72',
 '82',
 '83',
 '72',
 '79',
 '100',
 '74',
 '81',
 '83',
 '83',
 '65',
 '75',
 '75',
 '72',
 '74',
 '94',
 '64',
 '71',
 '84',
 '79',
 '78',
 '80',
 '74',
 '95',
 '70',
 '89',
 '67',
 '66',
 '96',
 '51',
 '67',
 '70',
 '92',
 '68',
 '81',
 '94',
 '69',
 '82',
 '85',
 '85',
 '59',
 '91',
 '76',
 '52',
 '88',
 '63',
 '94',
 '65',
 '64',
 '92',
 '75',
 '85',
 '76',
 '66',
 '71',
 '81',
 '85',
 '88',
 '81',
 '76',
 '89',
 '94',
 '78',
 '90',
 '79',
 '62',
 '90',
 '71',
 '73',
 '97',
 '72',
 '91',
 '77',
 '58',
 '87',
 '87',
 '81',
 '83',
 '84',
 '65',
 '82',
 '89',
 '61',
 '90',
 '82',
 '94',
 '65',
 '74',
 '91',
 '84',
 '68',
 '85',
 '78',
 '63',
 '78',
 '64',
 '77',
 '86',
 '77',
 '91',
 '70',
 '76',
 '69',
 '88',
 '78',
 '87',
 '82',
 '70',
 '62',
 '77',
 '80',
 '66',
 '69',
 '90',
 '68',
 '94',
 '59',
 '80',
 '68',
 '80',
 '84',
 '79',
 '75',
 '55',
 '66',
 '74',
 None,
 '81',
 '65',
 '62',
 '96',
 '68',
 '84',
 '81',
 '74'

In [10]:
#pandas dataframe        
movies = pd.DataFrame({
'movie': titles,
'year': years,
'timeMin': runtime,
'imdb': imdb_ratings,
'votes': votes,
'metascores': newMetalist,    
'us_grossMillions': us_gross,
'director': director_lists,
'stars': star_lists,
'genre': genres    
})

In [11]:
movies.shape

(1000, 10)

In [ ]:
# write dataFrame to CSV file -( creat file before then execute )
result.to_csv('IMDBMovieData.csv',index=False)